In [37]:
import random

import time

from typing import Callable, Any, Dict, List

In [36]:
# class Benchmark(Dict[str, List[int]]):
#     @staticmethod
#     def dt(func: Callable[..., Any]) -> Callable[..., Any]:
#         """ Decorador que calcula el tiempo que tarda en ejecutar la función."""
#         def wrapper(*args, **kwargs):
#             t_i = time.perf_counter()
#             result = func(*args, **kwargs)
#             dt_ = time.perf_counter() - t_i
#             print(f"{func.__name__}: {dt_}")
#             return result
#         return wrapper
# bench = Benchmark()

In [34]:
# @bench.dt
# def funcion_costosa():
#     time.sleep(random.randint(1,3))
# funcion_costosa()

funcion_costosa: 3.0017871449999802


In [44]:
import time
import random
from typing import List, NewType, Callable, Any

T_BenchName = NewType("bench_name", str)
T_dt = NewType("dt", int)
T_dts = NewType("dts", List[T_dt])


class Benchmark(Dict[T_BenchName, T_dts]):
    """ Benchmark de todas las funciones decoradas con `dt`."""
    def __init__(self, bench_names: List[str]):
        super().__init__({name: [] for name in bench_names})

    def dt(self, name: str) -> Callable[..., Any]:
        """
        Decorador: Calcula el tiempo que tarda en ejecutar la función decorada.
        - Se almacena dentro de `Benchmark` con key `name` en una lista.
        """
        def decorator(func: Callable[..., Any]) -> Callable[..., Any]:
            def wrapper(*args, **kwargs):
                t_i = time.perf_counter()
                result = func(*args, **kwargs)
                self[name].append(time.perf_counter() - t_i)
                return result
            return wrapper
        return decorator

bench = Benchmark(["hola", "funcion_costosa"])

@bench.dt(name="funcion_costosa")
def funcion_costosa():
    time.sleep(random.randint(1, 3))

for _ in range(5):
    funcion_costosa()


In [45]:
bench

{'hola': [],
 'funcion_costosa': [3.00134452400016,
  3.0030490560002363,
  1.0010558879994278,
  2.0017465700002504,
  2.0021595789994535]}